In [34]:
import numpy as np
import matplotlib.pyplot as plt

path = 'data_chb15_01.npy'
path2 = 'data_chb15_02.npy'
path3 = 'data_chb15_03.npy'
path4 = 'data_chb15_06.npy'
path5 = 'data_chb15_10.npy'

data = np.load(path)
data2 = np.load(path2)
data3 = np.load(path3)
data4 = np.load(path4)
data5 = np.load(path4)

data_add = np.append(data[0],data2[0])
data_add = np.append(data_add,data3[0])
data_add = np.append(data_add,data4[0])
data_add = np.append(data_add,data5[0])

label = np.append(data[2],data2[2])
label = np.append(label,data3[2])
label = np.append(label,data4[2])
label = np.append(label,data5[2])

print(data_add.shape, label.shape)

(4608512,) (4608512,)


In [35]:
import statistics as st

def strided_app(a, L, S ):  # Window len = L, Stride len/stepsize = S
    nrows = ((a.size-L)//S)+1
    n = a.strides[0]
    return np.lib.stride_tricks.as_strided(a, shape=(nrows,L), strides=(S*n,n))

x = strided_app(data_add, L = 256, S = 128)
# print(x.shape)

z = strided_app(label, L = 256, S = 128)
y = np.zeros(x.shape[0])
for i in range(0,x.shape[0]):
    if st.mode(z[i]) == 0:
        y[i] = 0
    elif st.mode(z[i]) == 1:
        y[i] = 1
    else:
        y[i] = 2
non = np.count_nonzero(y==0)
soz = np.count_nonzero(y==1)
ict = np.count_nonzero(y==2)
# print(y.shape)
print('set of label "0": {}'.format(non))
print('set of label "1": {}'.format(soz))
print('set of label "2": {}'.format(ict))

set of label "0": 35383
set of label "1": 120
set of label "2": 500


In [36]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Undersampling
from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import EditedNearestNeighbours

# enn = EditedNearestNeighbours()
# rus = RandomUnderSampler(random_state=0)
nr = NearMiss(n_neighbors=3)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)
X_train_resample, y_train_resample = nr.fit_resample(X_train, y_train)
X_test_resample, y_test_resample = nr.fit_resample(X_test, y_test)

# print('After Undersampling, the shape of train_X: {}'.format(X_train_resample.shape))
# print('After Undersampling, the shape of train_y: {} \n'.format(y_train_resample.shape))
# print("After Undersampling, counts of label '0': {}".format(sum(y_train_resample == 0)))
# print("After Undersampling, counts of label '1': {}".format(sum(y_train_resample == 1)))
# print("After Undersampling, counts of label '2': {}\n".format(sum(y_train_resample == 2)))

# print('After Undersampling, the shape of test_X: {}'.format(X_test_resample.shape))
# print('After Undersampling, the shape of test_y: {}'.format(y_test_resample.shape))
# print("After Undersampling, counts of label '0': {}".format(sum(y_test_resample == 0)))
# print("After Undersampling, counts of label '1': {}".format(sum(y_test_resample == 1)))
# print("After Undersampling, counts of label '2': {}".format(sum(y_test_resample == 2)))

# print(X_test.shape)
# print(y_test.shape)
# print('After Undersampling, the shape of train_X: {}'.format(X_train.shape))
# print('After Undersampling, the shape of train_y: {} \n'.format(y_train.shape))
# print("After Undersampling, counts of label '0': {}".format(sum(y_train == 0)))
# print("After Undersampling, counts of label '1': {}".format(sum(y_train == 1)))
# print("After Undersampling, counts of label '2': {}".format(sum(y_train == 2)))

In [37]:
# SVC resample_test
from sklearn.svm import SVC
svm_model_linear = SVC(kernel = 'rbf', C = 1).fit(X_train_resample, y_train_resample)
svm_predictions = svm_model_linear.predict(X_test_resample)

# model accuracy for X_test  
accuracy = svm_model_linear.score(X_test_resample, y_test_resample)
  
# creating a confusion matrix
cm = confusion_matrix(y_test_resample, svm_predictions)

print(accuracy)
print(cm)

# from sklearn.metrics import classification_report
# print('\nClassification Report\n')
# print(classification_report(y_test, svm_predictions, target_names=['non-seizure', 'soz', 'ictal']))


0.803921568627451
[[27  0  7]
 [ 3 27  4]
 [ 5  1 28]]


In [38]:
# SVC non-resample test
from sklearn.svm import SVC
from sklearn import metrics
svm_model_linear = SVC(kernel = 'rbf', C = 1).fit(X_train_resample, y_train_resample)
svm_predictions = svm_model_linear.predict(X_test)

# model accuracy for X_test  
accuracy = metrics.accuracy_score(y_test, svm_predictions)
# creating a confusion matrix
cm = confusion_matrix(y_test, svm_predictions)

print(X_test.shape,'----',y_test.shape)
print(accuracy)
print(cm) 
# 4313 6327 4480 (vertical) / 14240 238 642 (horizontal) => total = test.shape

(10801, 256) ---- (10801,)
0.07240070363855199
[[ 707 6866 3029]
 [   3   27    4]
 [   5  112   48]]


In [39]:
# KNN resample test
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
knn = KNeighborsClassifier(n_neighbors = 3).fit(X_train_resample, y_train_resample)
# K = sqrt(samples) or plot the error rate
knn_predictions = knn.predict(X_test_resample)
accuracy = metrics.accuracy_score(y_test_resample, knn_predictions)
cm = confusion_matrix(y_test_resample, knn_predictions)

print(accuracy)
print(cm)

# error_rate = []
# for i in range(1,50):
#  knn = KNeighborsClassifier(n_neighbors=i)
#  knn.fit(X_train_resample,y_train_resample)
#  pred_i = knn.predict(X_test_resample)
#  error_rate.append(np.mean(pred_i != y_test_resample))

# plt.figure(figsize=(10,6))
# plt.plot(range(1,50),error_rate,color='blue', linestyle='dashed', 
#          marker='o',markerfacecolor='red', markersize=10)
# plt.title('Error Rate vs. K Value')
# plt.xlabel('K')
# plt.ylabel('Error Rate')
#print("Minimum error:-",min(error_rate),"at K =",error_rate.index(min(error_rate)))

0.5784313725490197
[[31  0  3]
 [10 14 10]
 [20  0 14]]


In [40]:
# KNN non-resample test
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
knn = KNeighborsClassifier(n_neighbors = 26,).fit(X_train_resample, y_train_resample)
knn_predictions = knn.predict(X_test)
accuracy = metrics.accuracy_score(y_test, knn_predictions)
cm = confusion_matrix(y_test, knn_predictions)

print(accuracy)
print(cm)

0.6773446903064531
[[7271 1147 2184]
 [  22    1   11]
 [ 106   15   44]]


In [41]:
# decision tree
from sklearn.tree import DecisionTreeClassifier
dtree_model = DecisionTreeClassifier(max_depth = 2).fit(X_train_resample, y_train_resample)
dtree_predictions = dtree_model.predict(X_test_resample)
  
accuracy = metrics.accuracy_score(y_test_resample, dtree_predictions)
cm = confusion_matrix(y_test_resample, dtree_predictions)

print(accuracy)
print(cm)

0.45098039215686275
[[33  1  0]
 [21 13  0]
 [32  2  0]]


In [42]:
# random forest
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 100) 

clf.fit(X_train_resample, y_train_resample)
rand_predictions = clf.predict(X_test_resample)

accuracy = metrics.accuracy_score(y_test_resample, rand_predictions)
cm = confusion_matrix(y_test_resample, rand_predictions)

print(accuracy)
print(cm)

0.8627450980392157
[[26  0  8]
 [ 0 32  2]
 [ 4  0 30]]
